In [1]:
import sys
import json
import jsonlines
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

dirs = ["../..", ".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    get_model_tokenizer, get_template, get_vllm_engine, 
    inference_vllm, inference, VllmGenerationConfig, LoRARequest
)
from swift.tuners import Swift
from custom import CustomModelType, CustomTemplateType
import evaluation


def get_labels_preds(ckpt_dir, data_dir, data_type):

    with open(f"{ckpt_dir}/sft_args.json", "r") as f:
        sft_args = json.load(f)


    with jsonlines.open(f"{ckpt_dir}/../logging.jsonl", 'r') as f:
        for item in f.iter():
            training_result = item
            train_loss = training_result.get("train_loss")
            if train_loss is not None:
                break

    def get_engine_config_request(ckpt_dir):
        # 检查checkpoint是否为peft格式
        if os.path.exists(ckpt_dir + '/default'):
            raise Exception("只支持peft格式！")
            # if not os.path.exists(ckpt_dir + '-peft'):
            #     subprocess.run(["python", "../llm_export.py", "--ckpt", ckpt_dir, "--to_peft_format", "true"])
            # ckpt_dir = ckpt_dir + '-peft'


        lora_request = LoRARequest('default-lora', 1, ckpt_dir)

        model_type, template_type = sft_args["model_type"], sft_args["template_type"]
        vllm_engine = get_vllm_engine(
            model_type, 
            tensor_parallel_size=1,
            max_model_len=4096,
            enable_lora=True,
            max_loras=1, 
            max_lora_rank=8,
            engine_kwargs={
                "max_num_seqs": 128,
                "seed": 42,
            }
        )

        template = get_template(template_type, vllm_engine.hf_tokenizer)
        generation_config = VllmGenerationConfig(
            max_new_tokens=512,
            do_sample=False,
            temperature=0,
        )
        return vllm_engine, template, generation_config, lora_request

    def get_model_template():
        model_type, template_type = sft_args["model_type"], sft_args["template_type"]
        model, tokenizer = get_model_tokenizer(
            model_type, model_kwargs={'device_map': 'auto'},
            # model_dir=sft_args["model_cache_dir"],
            use_flash_attn=sft_args["use_flash_attn"]
        )
        model = Swift.from_pretrained(model, ckpt_dir, inference_mode=True)
        if sft_args["sft_type"] == 'adalora':
            model = model.to(model.dtype)
        model.generation_config.max_new_tokens = 512
        # model.generation_config.temperature = None
        model.generation_config.do_sample = False

        template = get_template(template_type, tokenizer)

        return model, template

    labels, preds = evaluation.cal_metric_single_llm(
        (get_model_template, get_engine_config_request), 
        (inference, inference_vllm), 
        sft_args, ckpt_dir, train_loss, save=True, use_vllm=False, 
        data_dir=data_dir, data_type=data_type,
    )

    return labels, preds



[INFO:swift] Successfully registered `/home/hanlv/workspace/code/research/infodemic/LLM/swift/swift/llm/data/dataset_info.json`


In [2]:
search_dates = [

    "2024-09-10",

]

ckpt_dir = "/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/output/covmis/Llama-3-8B-Instruct/with_llama3_info/brave/data1-split=8:2-ratio=1.0/dora-r=8/lr=9e-5-20240626-01:48:13/checkpoint-609"
data_type = "test" # test, valid

for search_date in search_dates:
    if os.path.exists(f"labels/{search_date}.json"):
        continue
    data_dir = f"/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/my_data/covmis/with_llama3_info/brave/train_valid_split/8:2/timeline_data1/test_data1_{search_date}.jsonl"

    labels, preds = get_labels_preds(ckpt_dir, data_dir, data_type)

    with open(f'labels/{search_date}.json', 'w') as f:
        json.dump(labels, f, indent=4)
    with open(f'preds/{search_date}.json', 'w') as f:
        json.dump(preds, f, indent=4)


[INFO:swift] Loading the model using model_dir: /home/css/models/Meta-Llama-3-8B-Instruct
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: LlamaConfig {
  "_name_or_path": "/home/css/models/Meta-Llama-3-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.42.4",
  "use_cache": true,
  "vocab_size": 128256
}

Special tokens have been added in the vocabulary, make sure the associated word embeddi

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[INFO:swift] model.max_model_len: 8192


Meta-Llama-3-8B-Instruct sft_type=dora lr=9e-5 train_loss=0.07930025


cnt_wrong=0:   0%|          | 0/1220 [00:00<?, ?it/s]                     /home/hanlv/miniconda3/envs/swift/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/hanlv/miniconda3/envs/swift/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
cnt_wrong=23: 100%|██████████| 1220/1220 [17:17<00:00,  1.18it/s]

{
    "model": "Meta-Llama-3-8B-Instruct",
    "train_test_split": "8:2",
    "train_ratio": "1.0",
    "train_loss": 0.07930025,
    "lr": "9e-5",
    "search_date": "2024-09-10",
    "ACC": 0.9811475409836066,
    "F1": 0.9740204188767674,
    "Precision": 0.985259172512592,
    "Recall": 0.9639130434782609
}
